## Is it Koyo? 

June 18, 2024 

By Natalie Ho 

After working on fast.ai's first module on classifying birds vs. dogs, I decided to see if I could make a module of my own identifying my friends, Koyo and Mitchell! 

In [4]:
#NB: Kaggle requires phone verification to use the internet or a GPU. If you haven't done that yet, the cell below will fail
#    This code is only here to check that your internet is enabled. It doesn't do anything else.
#    Here's a help thread on getting your phone number verified: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [5]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -U duckduckgo_search
    !pip install fastai
    !pip install fastbook

     |████████████████████████████████| 5.0 MB 11.2 MB/s            
     |████████████████████████████████| 97 kB 5.6 MB/s             
     |████████████████████████████████| 57 kB 4.2 MB/s             
     |████████████████████████████████| 2.8 MB 61.6 MB/s            
^C
ERROR: Operation cancelled by user
^C
Traceback (most recent call last):
  File "/opt/conda/bin/pip", line 6, in <module>
    from pip._internal.cli.main import main
  File "/opt/conda/lib/python3.7/site-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/opt/conda/lib/python3.7/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/opt/conda/lib/python3.7/site-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/opt/conda/lib/python3.7/site-packages/pip/_internal/cli/cmdoptions.py", l

## Step 1: Download images of Koyo

In [6]:
from fastbook import *
from fastai.vision.widgets import *
from fastai.vision.all import *

ModuleNotFoundError: No module named 'fastbook'

In [ ]:
'''
from fastdownload import download_url
dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=False)


im = Image.open(dest)
im.to_thumb(256,256)
'''

Opening a picture of Koyo using Pillow's Image class. 

In [ ]:
import pandas as pd
from PIL import Image

# Example of opening an image file
image_file_path = '/kaggle/input/new-images/Images/Koyo/IMG_4218 Small.jpeg'
image = Image.open(image_file_path).to_thumb(256,256)
image

## Step 2: Train our model

Used fast.ai's DataBlock to randomly select 80% of images to be the training set and 20% to be the validation set. This function shows a sample of 9 images that classifies if an image contains Mitchell or Koyo in it. 

**Challenges**
* Ran into an UnidentifiedImageError multiple times. I realized that I had to convert some images from HEIC into JPG. But even that wasn't enough! I had to compress all the images to avoid this error. 
* At first I had uploaded two separate folders of images. I realized I had to put the two Koyo and Mitchell subfolders into one larger Images folder as a complete dataset, so that DataBlock could be called and sort images of different categories. 

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders('/kaggle/input/new-images/Images/', bs=40)
#print(get_items)
dls.show_batch(max_n=9)


I thought one of the ways to fix the Unidentified image error was because the image was truncated, but turns out it was a different problem. 

Here I am fine tuning my model with resnet18, an image classification set pre-trained on ImageNet dataset. 

In [ ]:
!pip install pillow

from PIL import ImageFile

#ImageFile.LOAD_TRUNCATED_IMAGES = True


learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

## Step 3: Test the model

In [ ]:
test = '/kaggle/input/new-images/Images/Koyo/IMG_4594 Small.jpeg'
is_Koyo,_,probs = learn.predict(PILImage.create(test))

In [ ]:
print(f"This is a: {is_Koyo}.")
print(f"Probability it's a Koyo: {probs[0]:.4f}")
image = Image.open(test).to_thumb(256,256)
image

Step 4: Export 

In [ ]:
#export the architecture and trained parameters 
learn.export() 
#check that this export file exists with the ls method 
path = Path()
path.ls(file_exts='.pkl')

Why are koyo and mitchell flipped??

In [ ]:
#inference = using a model for getting predictions 
learn_inf = load_learner(path/'export.pkl')
learn_inf.predict(test)

#okay so this predict method and load learner works... just need to sort out the classify images function.

In [ ]:
#Hugging Face gradio needs to call a function 
categories = {'Koyo', 'Mitchell'}

def classify_image(img):
    prediction,index,probability = learn_inf.predict(img)
    print(prediction,index,probability)
    result = dict(zip(categories, map(float, probability)))
    print(result)
    return result

classify_image(test)

In [ ]:
!pip install gradio
import gradio as gr 

In [ ]:
#creating an interface
image = gr.inputs.Image(shape=(192,192))
label = gr.outputs.Label()
examples = ['/kaggle/input/new-images/Images/Koyo/IMG_8714 Small.jpeg', '/kaggle/input/new-images/Images/Mitchell/IMG_1199.jpeg']

interface = gr.Interface(fn=classify_image, inputs=image, outputs=label, examples=examples)
interface.launch(inline=False, share=True)

In [ ]:
#use IPython widgets to create an upload button 
#btn_upload = widgets.FileUpload()
#btn_upload

In [ ]:
'''
btn_upload = SimpleNamespace(data =[test])

#not sure what this actually does
img = PILImage.create(btn_upload.data[-1])
'''

In [ ]:
'''
#display image passed into upload 
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128, 128))
out_pl
'''

In [ ]:
'''
#get predictions 
pred,pred_idx,probs = learn_inf.predict(img)

#use Label to display prediction 
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]: .04f}'
lbl_pred
'''

In [ ]:
#create the classify button 
#button_run = widgets.Button(description='Go!')
#button_run

In [ ]:
#create a click event handler, which calls the function when pressed 'Go!'
'''
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

button_run.on_click(on_click_classify)
'''

In [ ]:
'''
btn_upload = widgets.FileUpload()
VBox([widgets.Label('Choose your player!'), btn_upload, button_run, out_pl, lbl_pred])

'''